In [1]:
import torch
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import cv2
import os
from skimage import img_as_bool, io, color, morphology
import math
from scipy import ndimage
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
import re
import pandas as pd

In [2]:
def scaled(d):
    a=d-d.min()
    b=d.max()-d.min()
    return np.divide(a, b, dtype="float32")#,out=np.zeros_like(a), where=b!=0)

def transcal(d):
    d = d.transpose(0,3,1,2)
    return scaled(d)

In [3]:
def normalize_by_reference(input_img, ref_img):
    # Get the minimum and maximum pixel values of the reference image
    ref_min = np.min(ref_img)
    ref_max = np.max(ref_img)

    # Get the minimum and maximum pixel values of the input image
    input_min = np.min(input_img)
    input_max = np.max(input_img)

    # Compute the scaling factors for the input image
    scale_min = (ref_min / input_min) if input_min > 0 else 1
    scale_max = ref_max / input_max

    # Apply the scaling factors to the input image
    normalized_img = input_img * scale_min
    normalized_img = np.where(normalized_img > ref_max, ref_max, normalized_img)
    
    return normalized_img

In [4]:
def lab_denoizer(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    A = cv2.fastNlMeansDenoising(a, None, 4, 7, 21)
    B = cv2.fastNlMeansDenoising(b, None, 4, 7, 21)
    filtered = cv2.fastNlMeansDenoising(l, None, 6, 7, 10)
    #noise = cv2.absdiff(l, filtered)
    #L = cv2.absdiff(l, noise)
    new=cv2.merge((filtered,A,B))
    new= cv2.cvtColor(new, cv2.COLOR_LAB2BGR)
    return new

In [5]:
from skimage import exposure

def im_hist_norm(src, ref_img):
    multi = True if src.shape[-1] > 1 else False
    matched = exposure.match_histograms(src, ref_img, multichannel=multi)
    return matched

In [6]:
def correct_overexposure(imag, channel="green", target_value=210):
    # Get the channel index
    channel_idx = {'blue': 0, 'green': 1, 'red': 2}[channel.lower()]

    # Split the image channels
    #b, g, r = cv2.split(imag)

    # Threshold the selected channel to identify overexposed pixels
    overexposed_pixels = imag[:,:,channel_idx] > target_value

    # Replace the overexposed pixels with the target value
    imag[:,:,channel_idx][overexposed_pixels] = target_value

    # Merge the channels back into a single image
    #normalized_img = cv2.merge([b, g, r])

    return imag #normalized_img

In [7]:
def calculate_rootlen(r,mmcoef_=1, intercept=0):
    a = [] # list of lengths 
    for i in range(r.shape[0]):
        image = np.round(r[i,0,:,:],decimals=0) #y[i,0,:,:]
        #image = cv2.resize(image,(256,256))
        image = img_as_bool(image)
        image = ndimage.binary_closing(image, structure=np.ones((3,3)))
        skeleton = morphology.medial_axis(image)
        sk = np.round(skeleton.sum(), decimals=1)
        ## Correction
        sk = np.round(sk*mmcoef_)+intercept
        #print(sk)
        a.append(sk)
    return a

In [8]:
def calculate_area(r,mmcoef_=1, intercept=0):
    a = [] # list of lengths 
    for i in range(r.shape[0]):
        image = np.round(r[i,0,:,:],decimals=0) #y[i,0,:,:]
        #image = cv2.resize(image,(256,256))
        image = img_as_bool(image)
        image = ndimage.binary_closing(image, structure=np.ones((3,3)))
        
        sk = np.round(image.sum(), decimals=1)
        ## Correction
        sk = np.round(sk*mmcoef_)+intercept
        #print(sk)
        a.append(sk)
    return a

In [9]:
def predict_mask(Model,dat):
    out=torch.tensor([])
    for i in range(0,dat.shape[0],1):
        torch.cuda.empty_cache() 
        j=i+1
        #print(j)
        if j==(dat.shape[0]+1):
            print(j)
            with torch.no_grad():
                outputs = Model(torch.from_numpy(dat[i:]).cuda()) 
            out = torch.cat((out,outputs.cpu()),0)
        else:
            with torch.no_grad():
                outputs = Model(torch.from_numpy(dat[i:j]).cuda()) 
            out = torch.cat((out,outputs.cpu()),0)
    out = out.cpu().detach().numpy()
    return out

In [10]:
model = torch.load("./BipEModel_gray")

C:\ProgramData\Anaconda3\envs\Image_processing\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] The specified module could not be found
  warn(f"Failed to load image Python extension: {e}")


In [11]:
model1 = torch.load("./BipEModel3.6")

In [12]:
main_dir="D:/Imaging_PY/Experiments/Biplabi_images/GO_TUBES/"
tubes = os.listdir(main_dir)

In [13]:
for tube in tubes:
    file_count = 0
    for _, _, files in os.walk(main_dir+tube+"/"):
        file_count += len(files)
    if file_count == 416:
        dir0 = main_dir+tube+"/"
        l = os.listdir(dir0)
        date_pattern = r"\d{4}\.\d{2}\.\d{2}"  # Regular expression pattern for date in format YYYY.MM.DD
        unique_dates = set()

        for file_name in l:
            match = re.search(date_pattern, file_name)
            if match:
                date = match.group()
                unique_dates.add(date)

        dates_list = list(unique_dates)
        dates_list = sorted(dates_list)
        # reference building
        print("reference building...")
        pattern = r"_L(\d{3})"
        ref={}
        try:
            for file_name in l:
                if dates_list[2] in file_name:
                    match = re.search(pattern, file_name)
                    #print(match)
                    img = cv2.imread(dir0+file_name)
                    img = lab_denoizer(np.flip(img,2))
                    img = cv2.resize(img, (640, 480))
                    location = match.group(1)
                    ref[location] = img
        except:
            print(tube+" is wrong")


        date_counts = {}
        pattern = r"_L(\d{3})"
        for date in dates_list:
            print(date)
            # building datset with same date
            X = np.zeros(shape=(1,480,640,3), dtype="uint8") #shape=(1,480,640,3)
            Grays = np.zeros(shape=(1,480,640,1), dtype="uint8")
            for file_name in l:
                if date in file_name:
                    match = re.search(pattern, file_name)
                    location = match.group(1)
                    img = cv2.imread(dir0+file_name)
                    img = lab_denoizer(np.flip(img,2))
                    img = cv2.resize(img, (640, 480))
                    im = im_hist_norm(img, ref[location])
                    # normalization
                    correct = correct_overexposure(im,"green",target_value=210)
                    correct = correct_overexposure(correct,"blue",target_value=210)
                    correct = correct_overexposure(correct,"red",target_value=210)
                    X = np.append(X, [correct], axis=0)
                    # gray   
                    gray = cv2.cvtColor(correct, cv2.COLOR_BGR2GRAY)
                    gray = np.expand_dims(gray, axis=2)
                    Grays = np.append(Grays, [gray], axis=0)
                    
                    

            X = X[1:]
            Grays = Grays[1:]
            X_test = transcal(X)
            G_test = transcal(Grays)

            # prediction of masks
            print("predicting masks...")
            out1 = predict_mask(model1,X_test)
            out2 = predict_mask(model, G_test)
            out3 = scaled(out1+out2)
            out3 = (out3>0.3).astype("float32")

            # claculate lengths and add to date_counts dictionary
            total_lengths=calculate_rootlen(out3)  
            total_area = calculate_area(out3)
            date_counts[date] = [total_lengths,total_area]

        lengths = {}
        areas = {}

        for key, value in date_counts.items():
            list1, list2 = value
            lengths[key] = list1
            areas[key] = list2

        L = pd.DataFrame.from_dict(lengths)
        A = pd.DataFrame.from_dict(areas)

        L.to_csv(tube+"_length.csv")
        A.to_csv(tube+"_areas.csv")
        print(tube)
    else:
        print(tube+" different number of pictures")
        pass
    
    

reference building...
2020.05.30


C:\ProgramData\Anaconda3\envs\Image_processing\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: `multichannel` is a deprecated argument name for `match_histograms`. It will be removed in version 1.0. Please use `channel_axis` instead.
  """


predicting masks...
2020.06.24
predicting masks...
2020.07.16
predicting masks...
2020.08.12
predicting masks...
2020.09.22
predicting masks...
2020.10.08
predicting masks...
2020.12.20
predicting masks...
2021.03.24
predicting masks...
2021.04.20
predicting masks...
2021.06.17
predicting masks...
2021.07.14
predicting masks...
2021.08.03
predicting masks...
2021.12.23
predicting masks...
2022.04.13
predicting masks...
2022.06.20
predicting masks...
2022.08.19
predicting masks...
GO-Tube 23


In [15]:
for date in dates_list:
    print(date)

2020.05.26
2020.06.25
2020.07.17
2020.08.12
2020.09.18
2020.10.07
2020.11.24
2021.03.12
2021.04.18
2021.06.20
2021.07.13
2021.08.02
2022.01.13
2022.04.11
2022.06.16
2022.08.18


In [22]:
dir0 = main_dir+"GO-Tube 19"+"/"
l = os.listdir(dir0)
date_pattern = r"\d{4}\.\d{2}\.\d{2}"  # Regular expression pattern for date in format YYYY.MM.DD
unique_dates = set()
for file_name in l:
            match = re.search(date_pattern, file_name)
            if match:
                date = match.group()
                unique_dates.add(date)
dates_list = list(unique_dates)

In [23]:
sorted(dates_list)

['2020.05.30',
 '2020.06.24',
 '2020.07.17',
 '2020.08.12',
 '2020.09.22',
 '2020.10.08',
 '2020.12.20',
 '2021.03.24',
 '2021.04.20',
 '2021.06.17',
 '2021.07.14',
 '2021.08.03',
 '2021.12.23',
 '2022.04.12',
 '2022.06.20',
 '2022.08.19']

In [24]:
for tube in tubes:
    file_count = 0
    for _, _, files in os.walk(main_dir+tube+"/"):
        file_count += len(files)
    print(f"Number of files in {tube}: {file_count}")

Number of files in GO-Tube 19: 416
Number of files in GO-tube 20: 416
Number of files in GO-Tube 21: 416
Number of files in GO-Tube 22: 416
Number of files in GO-Tube 23: 289
Number of files in GO-Tube 24: 416
Number of files in GO-Tube 25: 416
Number of files in GO-Tube 26: 390
Number of files in GO-Tube 27: 416
Number of files in GO-Tube 28: 416
Number of files in GO-Tube 29: 416
Number of files in GO-Tube 30: 416
Number of files in GO-Tube 31: 416
Number of files in GO-Tube 32: 416
Number of files in GO-Tube 33: 416
Number of files in GO-Tube 34: 416
Number of files in GO-Tube 35: 416
Number of files in GO-Tube 36: 416
Number of files in GO-Tube 37: 416
Number of files in GO-Tube 38: 416
Number of files in GO-Tube 39: 390
Number of files in GO-Tube 40: 416
Number of files in GO-Tube 41: 416
Number of files in GO-Tube 42: 416
Number of files in GO-Tube 43: 416
Number of files in GO-Tube 44: 416
Number of files in GO-Tube 45: 416
Number of files in GO-Tube 46: 416
Number of files in G